<a href="https://colab.research.google.com/github/mj950425/kaggle_transcription/blob/main/toxic_classification_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"minjoon1324","key":"580bbb6318c57e42ab0f48e2487e7d3a"}'}

In [2]:
!ls -1ha kaggle.json

kaggle.json


In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# Permission Warning 이 일어나지 않도록 
!chmod 600 ~/.kaggle/kaggle.json
# 본인이 참가한 모든 대회 보기 
!kaggle competitions list

ref                                            deadline             category            reward  teamCount  userHasEntered  
---------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
contradictory-my-dear-watson                   2030-07-01 23:59:00  Getting Started     Prizes        102           False  
gan-getting-started                            2030-07-01 23:59:00  Getting Started     Prizes        209           False  
tpu-getting-started                            2030-06-03 23:59:00  Getting Started  Knowledge        493           False  
digit-recognizer                               2030-01-01 00:00:00  Getting Started  Knowledge       3398           False  
titanic                                        2030-01-01 00:00:00  Getting Started  Knowledge      27247            True  
house-prices-advanced-regression-techniques    2030-01-01 00:00:00  Getting Started  Knowledge       6976           False  
connectx

In [4]:
!kaggle competitions download -c jigsaw-multilingual-toxic-comment-classification

  0% 0.00/3.44M [00:00<?, ?B/s]
100% 3.44M/3.44M [00:00<00:00, 114MB/s]
 99% 645M/650M [00:04<00:00, 161MB/s]
100% 650M/650M [00:04<00:00, 141MB/s]
  0% 0.00/1.35M [00:00<?, ?B/s]
100% 1.35M/1.35M [00:00<00:00, 89.9MB/s]
 83% 66.0M/79.6M [00:00<00:00, 118MB/s]
100% 79.6M/79.6M [00:00<00:00, 132MB/s]
 64% 8.00M/12.4M [00:00<00:00, 81.0MB/s]
100% 12.4M/12.4M [00:00<00:00, 79.2MB/s]
 95% 277M/292M [00:02<00:00, 139MB/s]
100% 292M/292M [00:02<00:00, 132MB/s]
 67% 25.0M/37.3M [00:00<00:00, 57.6MB/s]
100% 37.3M/37.3M [00:00<00:00, 122MB/s] 
 80% 24.0M/29.8M [00:00<00:00, 75.4MB/s]
100% 29.8M/29.8M [00:00<00:00, 118MB/s] 
  0% 0.00/612k [00:00<?, ?B/s]
100% 612k/612k [00:00<00:00, 171MB/s]


In [5]:
!unzip jigsaw-toxic-comment-train.csv.zip
!unzip jigsaw-toxic-comment-train-processed-seqlen128.csv.zip
!unzip jigsaw-unintended-bias-train.csv.zip
!unzip jigsaw-unintended-bias-train-processed-seqlen128.csv.zip
!unzip kaggle.json
!unzip sample_data
!unzip sample_submission.csv
!unzip test.csv.zip
!unzip test-processed-seqlen128.csv.zip
!unzip validation.csv.zip
!unzip validation-processed-seqlen128.csv.zip

Archive:  jigsaw-toxic-comment-train.csv.zip
  inflating: jigsaw-toxic-comment-train.csv  
Archive:  jigsaw-toxic-comment-train-processed-seqlen128.csv.zip
  inflating: jigsaw-toxic-comment-train-processed-seqlen128.csv  
Archive:  jigsaw-unintended-bias-train.csv.zip
  inflating: jigsaw-unintended-bias-train.csv  
Archive:  jigsaw-unintended-bias-train-processed-seqlen128.csv.zip
  inflating: jigsaw-unintended-bias-train-processed-seqlen128.csv  
Archive:  kaggle.json
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of kaggle.json or
        kaggle.json.zip, and cannot find kaggle.json.ZIP, period.
unzip:  cannot find or open sample_data, sample_data.zip or sample_data.ZIP.
Archive:  sample_submission.csv
  End-of-central-directory signat

In [6]:
!pip install transformers
!pip install sentencepiece

     |████████████████████████████████| 2.0MB 5.5MB/s 
     |████████████████████████████████| 3.2MB 23.4MB/s 
     |████████████████████████████████| 890kB 34.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=698d0044c245bc964f4da495006fdc2b8e2b95260f5d33dbef770ba8ab8f5db2
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 1.2MB 5.0MB/s 


In [7]:
!pip install kaggle_datasets

ERROR: Could not find a version that satisfies the requirement kaggle_datasets (from versions: none)
ERROR: No matching distribution found for kaggle_datasets


In [8]:
from kaggle_datasets import KaggleDatasets


ModuleNotFoundError: ignored

In [9]:
import os
import gc

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import transformers
from transformers import TFAutoModel, AutoTokenizer
from tqdm.notebook import tqdm
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors

In [10]:
def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts, 
        return_attention_mask=False, 
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen,
        truncation=True
    )
    
    return np.array(enc_di['input_ids'])




In [33]:
def dict_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        pd.DataFrame(texts), 
        return_attention_mask=True, 
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen,
        truncation=True
    )
    
    return {
        "input_ids": np.array(enc_di['input_ids']),
        "attention_mask": np.array(enc_di['attention_mask'])
    }

In [12]:
def build_model(transformer, max_len=512):
    """
    https://www.kaggle.com/xhlulu/jigsaw-tpu-distilbert-with-huggingface-and-keras
    """
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
    attention_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
    sequence_output = transformer({"input_ids": input_word_ids, "attention_mask": attention_mask})[0]
    cls_token = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(cls_token)
    
    model = Model(inputs={
        "input_ids": input_word_ids,
        "attention_mask": attention_mask
    }, outputs=out)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [13]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.1.60.34:8470


INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Initializing the TPU system: grpc://10.1.60.34:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.1.60.34:8470


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


In [14]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

In [15]:
MAX_LEN = 192
MODEL = 'allenai/unifiedqa-t5-small'
AUTO = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 4 * strategy.num_replicas_in_sync
tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [50]:
train1 = pd.read_csv("jigsaw-toxic-comment-train.csv")
train2 = pd.read_csv("jigsaw-unintended-bias-train.csv")
train2.toxic = train2.toxic.round().astype(int)

valid = pd.read_csv('validation.csv').sample(frac=1.0)
n_valid_steps = valid.shape[0] // BATCH_SIZE
test = pd.read_csv('test.csv')
sub = pd.read_csv('sample_submission.csv')




In [51]:
# Combine train1 with a subset of train2
train = pd.concat([
    train1[['comment_text', 'toxic']],
#     train2[['comment_text', 'toxic']].query('toxic==1'),
#     train2[['comment_text', 'toxic']].query('toxic==0').sample(n=100000, random_state=0)
]).sample(frac=1.0)
n_train_steps = train.shape[0] // BATCH_SIZE
del train1, train2
gc.collect()

112

In [42]:
type(train.comment_text.values)

numpy.ndarray

In [43]:
tokenizer

PreTrainedTokenizerFast(name_or_path='allenai/unifiedqa-t5-small', vocab_size=32100, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>', '<extra_id_

In [53]:
# %%time 

x_train = dict_encode(train.comment_text.values, tokenizer, maxlen=MAX_LEN)
y_train = train.toxic.values

del train
gc.collect()

x_valid = dict_encode(valid.comment_text.values, tokenizer, maxlen=MAX_LEN)
y_valid = valid.toxic.values
del valid
gc.collect()
x_test = dict_encode(test.content.values, tokenizer, maxlen=MAX_LEN)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


<PrefetchDataset shapes: ({input_ids: (None, 192), attention_mask: (None, 192)}, (None,)), types: ({input_ids: tf.int64, attention_mask: tf.int64}, tf.int64)>

In [55]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(4096)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_valid, y_valid))
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(x_test)
    .batch(BATCH_SIZE)
)
del x_train, x_valid, y_train, y_valid
gc.collect()

0

In [ ]:
!pip install transfor

In [63]:
MODEL = 'bert-base-uncased'
with strategy.scope():
    transformer_layer = TFAutoModel.from_pretrained(MODEL)
    model = build_model(transformer_layer, max_len=MAX_LEN)
model.summary()

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
attention_mask (InputLayer)     [(None, 192)]        0                                            
__________________________________________________________________________________________________
input_ids (InputLayer)          [(None, 192)]        0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     TFBaseModelOutputWit 109482240   attention_mask[0][0]             
                                                                 input_ids[0][0]                  
__________________________________________________________________________________________________
tf.__operators__.getitem (Slici (None, 768)          0           tf_bert_model[0][0]          

In [64]:
# Configuration
EPOCHS = 2

train_history = model.fit(
    train_dataset,
    steps_per_epoch=n_train_steps,
    validation_data=valid_dataset,
    epochs=EPOCHS
)

Epoch 1/2


6985/6985 [==============================] - ETA: 0s - loss: 0.1984 - accuracy: 0.9322WARNING:tensorflow:The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


6985/6985 [==============================] - 713s 92ms/step - loss: 0.1984 - accuracy: 0.9322 - val_loss: 0.4596 - val_accuracy: 0.8258
Epoch 2/2
6985/6985 [==============================] - 633s 91ms/step - loss: 0.1180 - accuracy: 0.9541 - val_loss: 0.4771 - val_accuracy: 0.8256


In [65]:
train_history_2 = model.fit(
    valid_dataset.repeat(),
    steps_per_epoch=n_valid_steps,
    epochs=EPOCHS
)

Epoch 1/2
250/250 [==============================] - 23s 90ms/step - loss: 0.3896 - accuracy: 0.8473
Epoch 2/2
250/250 [==============================] - 23s 90ms/step - loss: 0.3159 - accuracy: 0.8633


In [ ]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

In [66]:
sub['toxic'] = model.predict(test_dataset, verbose=1)
sub.to_csv('submission.csv', index=False)

1995/1995 [==============================] - 60s 28ms/step


In [67]:
model.save_weights("final_weights.h5")